# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Load information abou the api
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
api_key = weather_api_key

# Build partial url
query_url = f"{url}appid={api_key}&units={units}&q="

# Create dataframe and add columns
cities_df = pd.DataFrame({'City':cities})
cities_df['Cloudiness'] = ""
cities_df['Country'] = ""
cities_df['Date'] = ""
cities_df['Humidity'] = ""
cities_df['Lat'] = ""
cities_df['Lng'] = ""
cities_df['Max Temp'] = ""
cities_df['Wind Speed'] = ""

# Use iterrows to iterate through cities dataframe
for index, row in cities_df.iterrows():

    # get restaurant type from df
    city = row['Cities']

    # assemble url and make API request   
    response = requests.get(query_url+city).json()

    try:     
        cities_df.loc[index,'Cloudiness'] = response['clouds']['all']
        cities_df.loc[index,'Country'] = response['sys']['country']
        cities_df.loc[index,'Date'] = response['dt']
        cities_df.loc[index,'Humidity'] = response['main']['humidity']
        cities_df.loc[index,'Lat'] = response['coord']['lat']
        cities_df.loc[index,'Lng'] = response['coord']['lon']
        cities_df.loc[index,'Max Temp'] = response['main']['temp_max']
        cities_df.loc[index,'Wind Speed'] = response['wind']['speed']
        print(f"Retrived information for country {response['id']} | {response['name']}.")
    
    except (KeyError, IndexError):
        print("{city} not found. Skipping.")
        
    print("------------")


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
cities_df.count()

In [ ]:
cities_df.to_csv('cities.csv')
cities_df

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
#  Change values of cities dataframe to numeric to plot them
cities_df['Cloudiness'] = pd.to_numeric(cities_df['Cloudiness'])
cities_df['Humidity'] = pd.to_numeric(cities_df['Humidity'])
cities_df['Lat'] = pd.to_numeric(cities_df['Lat'])
cities_df['Lng'] = pd.to_numeric(cities_df['Lng'])
cities_df['Max Temp'] = pd.to_numeric(cities_df['Max Temp'])
cities_df['Wind Speed'] = pd.to_numeric(cities_df['Wind Speed'])

# Declare variables
clouds = cities_df['Cloudiness']
humidity = cities_df['Humidity']
lat = cities_df['Lat']
lng = cities_df['Lng']
max_temp = cities_df['Max Temp']
wind_speed = cities_df['Wind Speed']

In [ ]:
plt.scatter(lat,max_temp, edgecolors='black',c='red')
plt.title('City Latitude vs. Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature Celcius')
plt.grid()
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
plt.scatter(lat,humidity, edgecolors='black',c='navy')
plt.title('City Latitude vs. Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity %')
plt.grid()
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(lat,clouds, edgecolors='black',c='gray')
plt.title('City Latitude vs. Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.grid()
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(lat,wind_speed, edgecolors='black',c='white')
plt.title('City Latitude vs. Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.grid()
plt.show()

## Linear Regression

### OPTIONAL: Create a function to create Linear Regression plots


#### Create Northern and Southern Hemisphere DataFrames


In [ ]:
# Separe the hemisphere
north = cities_df.loc[cities_df['Lat']>=0]
south = cities_df.loc[cities_df['Lat']<0]

# Declare south variables
sclouds = south['Cloudiness']
shumidity = south['Humidity']
slat = south['Lat']
slng = south['Lng']
smax_temp = south['Max Temp']
swind_speed = south['Wind Speed']

# Declare north variables
nclouds = north['Cloudiness']
nhumidity = north['Humidity']
nlat = north['Lat']
nlng = north['Lng']
nmax_temp = north['Max Temp']
nwind_speed = north['Wind Speed']

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
nmaxtemp_slope, nmaxtemp_int, nmaxtemp_r, nmaxtemp_p, nmaxtemp_std_err = st.linregress(nlat, nmax_temp)
nmaxtemp_fit = nmaxtemp_slope * nlat + nmaxtemp_int
line_eq = f'y = {str(round(nmaxtemp_slope,2))} x + {str(round(nmaxtemp_int,2))}'

print(f'The r-squared is: {round(nmaxtemp_r,3)}')

plt.scatter(nlat,nmax_temp, edgecolors='black',c='red')
plt.plot(nlat,nmaxtemp_fit,"b-")
plt.title('Northern Hemisphere: City Latitude vs. Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature Celcius')
plt.annotate(line_eq,(4,-26),fontsize=12,c='blue')
plt.grid()
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
smaxtemp_slope, smaxtemp_int, smaxtemp_r, smaxtemp_p, smaxtemp_std_err = st.linregress(slat, smax_temp)
smaxtemp_fit = smaxtemp_slope * slat + smaxtemp_int
line_eq = f'y = {str(round(smaxtemp_slope,2))} x + {str(round(smaxtemp_int,2))}'

print(f'The r-squared is: {round(smaxtemp_r,3)}')

plt.scatter(slat,smax_temp, edgecolors='black',c='red')
plt.plot(slat,smaxtemp_fit,"b-")
plt.title('Southern Hemisphere: City Latitude vs. Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature Celcius')
plt.annotate(line_eq,(-25,7),fontsize=12,c='blue')
plt.grid()
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
nhumidity_slope, nhumidity_int, nhumidity_r, nhumidity_p, nhumidity_std_err = st.linregress(nlat, nhumidity)
nhumidity_fit = nhumidity_slope * nlat + nhumidity_int
line_eq = f'y = {str(round(nhumidity_slope,2))} x + {str(round(nhumidity_int,2))}'

print(f'The r-squared is: {round(nhumidity_r,3)}')

plt.scatter(nlat,nhumidity, edgecolors='black',c='navy')
plt.plot(nlat,nhumidity_fit,"r-")
plt.title('Northern Hemisphere: City Latitude vs. Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.annotate(line_eq,(45,15),fontsize=12,c='red')
plt.grid()
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
shumidity_slope, shumidity_int, shumidity_r, shumidity_p, shumidity_std_err = st.linregress(slat, shumidity)
shumidity_fit = shumidity_slope * slat + shumidity_int
line_eq = f'y = {str(round(shumidity_slope,2))} x + {str(round(shumidity_int,2))}'

print(f'The r-squared is: {round(shumidity_r,3)}')

plt.scatter(slat,shumidity, edgecolors='black',c='navy')
plt.plot(slat,shumidity_fit,"r-")
plt.title('Southern Hemisphere: City Latitude vs. Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.annotate(line_eq,(-55,15),fontsize=12,c='red')
plt.grid()
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
nclouds_slope, nclouds_int, nclouds_r, nclouds_p, nclouds_std_err = st.linregress(nlat, nclouds)
nclouds_fit = nclouds_slope * nlat + nclouds_int
line_eq = f'y = {str(round(nclouds_slope,2))} x + {str(round(nclouds_int,2))}'

print(f'The r-squared is: {round(nclouds_r,3)}')

plt.scatter(nlat,nclouds, edgecolors='black',c='gray')
plt.plot(nlat,nhumidity_fit,"g-")
plt.title('Northern Hemisphere: City Latitude vs. Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.annotate(line_eq,(45,12),fontsize=12,c='red')
plt.grid()
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
sclouds_slope, sclouds_int, sclouds_r, sclouds_p, sclouds_std_err = st.linregress(slat, sclouds)
sclouds_fit = sclouds_slope * slat + sclouds_int
line_eq = f'y = {str(round(sclouds_slope,2))} x + {str(round(sclouds_int,2))}'

print(f'The r-squared is: {round(sclouds_r,3)}')

plt.scatter(slat,sclouds, edgecolors='black',c='gray')
plt.plot(slat,sclouds_fit,"g-")
plt.title('Southern Hemisphere: City Latitude vs. Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.annotate(line_eq,(-55,13),fontsize=12,c='red')
plt.grid()
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
nwind_speed_slope, nwind_speed_int, nwind_speed_r, nwind_speed_p, nwind_speed_std_err = st.linregress(nlat, nwind_speed)
nwind_speed_fit = nwind_speed_slope * nlat + nwind_speed_int
line_eq = f'y = {str(round(nwind_speed_slope,2))} x + {str(round(nwind_speed_int,2))}'

print(f'The r-squared is: {round(nwind_speed_r,3)}')

plt.scatter(nlat,nwind_speed, edgecolors='black',c='white')
plt.plot(nlat,nwind_speed_fit,"b-")
plt.title('Northern Hemisphere: City Latitude vs. Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.annotate(line_eq,(5,13),fontsize=12,c='red')
plt.grid()
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
swind_speed_slope, swind_speed_int, swind_speed_r, swind_speed_p, swind_speed_std_err = st.linregress(slat, swind_speed)
swind_speed_fit = swind_speed_slope * slat + swind_speed_int
line_eq = f'y = {str(round(swind_speed_slope,2))} x + {str(round(swind_speed_int,2))}'

print(f'The r-squared is: {round(swind_speed_r,3)}')

plt.scatter(slat,swind_speed, edgecolors='black',c='white')
plt.plot(slat,swind_speed_fit,"b-")
plt.title('Southern Hemisphere: City Latitude vs. Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.annotate(line_eq,(-50,13),fontsize=12,c='red')
plt.grid()
plt.show()